In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [7]:
wpp = pd.read_excel("data/WPP2022_SA3_GEN_F01_DEMOGRAPHIC_INDICATORS.xlsx")
print(wpp.columns.tolist())

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [12]:
WHO_standard = pd.read_csv("data/WHO_World_Standard_from_NIH_NCI.csv", usecols=['Age Group', 'WHO World Standard (%)'], skiprows=1, skipfooter=2, engine='python')
print(WHO_standard)

   Age Group  WHO World Standard (%)
0        0-4                   8.860
1        5-9                   8.690
2      10-14                   8.600
3      15-19                   8.470
4      20-24                   8.220
5      25-29                   7.930
6      30-34                   7.610
7      35-39                   7.150
8      40-44                   6.590
9      45-49                   6.040
10     50-54                   5.370
11     55-59                   4.550
12     60-64                   3.720
13     65-69                   2.960
14     70-74                   2.210
15     75-79                   1.520
16     80-84                   0.910
17     85-89                   0.440
18     90-94                   0.150
19     95-99                   0.040
20      100+                   0.005


In [5]:
copd = pd.read_csv("data/COPD_age-specific_USA_Uganda_2019.csv", skiprows=1, skipfooter=2, engine='python')
print(copd)

   Age group (years)  Death rate, United States, 2019  \
0                0-4                             0.04   
1                5-9                             0.02   
2              10-14                             0.02   
3              15-19                             0.02   
4              20-24                             0.06   
5              25-29                             0.11   
6              30-34                             0.29   
7              35-39                             0.56   
8              40-44                             1.42   
9              45-49                             4.00   
10             50-54                            14.13   
11             55-59                            37.22   
12             60-64                            66.48   
13             65-69                           108.66   
14             70-74                           213.10   
15             75-79                           333.06   
16             80-84           